# Direction reconstruction

Description here

## Imports

In [ ]:
import pandas

## Functions

### Load data with pandas

In [ ]:
def load_reset_dl1_pandas(indir = "./", fileName = "dl1_tail_gamma_z20_az180_LaPalma_baseline_run100_withMono.h5", config="test"):
    """(Re)load the file containing DL1(a) data and extract the data per telescope type."""
    # load DL1 images
    data_LST = pandas.read_hdf(f"{indir}/{fileName}", "/feature_events_LSTCam")
    data_MST = pandas.read_hdf(f"{indir}/{fileName}", "/feature_events_NectarCam")
    suffix = config # all generated plots will have this as a suffix in their name
    return data_LST, data_MST, suffix

### Quantities to plot

In [ ]:
def dl1_quantities(data):
    """A dictionary of the quantities available with this format of DL1 in protopipe.
    
    WARNING: for the moment protopipe uses one cleaning algorithm (biggest cluster),
    even though it allows for two;
    this means that all the quantities with the suffix "_reco" are the same as those without suffix.
    """
    
    if type(data)!=pandas.core.frame.DataFrame:
        
        dictionary = {

            "Intensity [#phe]"   : data.col("sum_signal_cam"), # aka SIZE
            "Width [m]"          : data.col("width"),
            "Length [m]"         : data.col("length"),
            "Skewness"           : data.col("skewness"),
            "Kurtosis"           : data.col("kurtosis"),
            "H_max [m]"          : data.col("h_max"),
            "n_pixel"            : data.col("n_pixel"),
            "Ellipticity"        : data.col("ellipticity"),
#             "Leakage 1"          : data.col("leak1_reco")  # see cta-observatory/protopipe#41
            "psi"                : (data.col("psi_reco") * u.deg).to(u.rad),
            "cog_x"              : data.col("cog_x"),
            "cog_y"              : data.col("cog_y"),

        }
        
    else:
        
        dictionary = {

            "Intensity [#phe]"   : data["sum_signal_cam"], # aka SIZE
            "Width [m]"          : data["width"],
            "Length [m]"         : data["length"],
            "Skewness"           : data["skewness"],
            "Kurtosis"           : data["kurtosis"],
            "H_max [m]"          : data["h_max"],
            "n_pixel"            : data["n_pixel"],
            "Ellipticity"        : data["ellipticity"],
    #         "Leakage 1"          : data.col("leak1_reco")  # see cta-observatory/protopipe#41
            "psi"                : data["psi_reco"],
            "cog_x"              : data["cog_x"],
            "cog_y"              : data["cog_y"],            

        }
    
    return dictionary

### Add statistical information to a plot

In [ ]:
def add_stats(x, ax):
    """Add a textbox containing statistical information."""
    mu = x.mean()
    median = np.median(x)
    sigma = x.std()
    textstr = '\n'.join((
        r'$\mu=%.2f$' % (mu, ),
        r'$\mathrm{median}=%.2f$' % (median, ),
        r'$\sigma=%.2f$' % (sigma, )))

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    # place a text box in upper left in axes coords
    ax.text(0.70, 0.85, 
            textstr, 
            transform=ax.transAxes, 
            fontsize=10,
            horizontalalignment='left',
            verticalalignment='center', 
            bbox=props)

## Load data

In [ ]:
# fill with the correct path, filename of the generated file in your system
data_LST, data_MST, config = load_reset_dl1_pandas()
cameras = ["LSTCam", "NectarCam"]

In [ ]:
# Get DL1 quantities as numpy arrays or pandas.Dataframe columns
DL1_LST = dl1_quantities(data_LST)
DL1_MST = dl1_quantities(data_MST)
DL1 = [DL1_LST, DL1_MST]

### Add secondary variables

In [ ]:
# add miss**2 to DL1 dictionaries in pandas format
for camera_index in range(len(cameras)):
    DL1[camera_index]["miss2 [deg**2]"] = miss_deg[camera_index]**2

## Plots and benchmarks

First we check if a _plots_ folder exists already.  
If not, we create it.

In [ ]:
Path("./plots_direction_reconstruction").mkdir(parents=True, exist_ok=True)

### Direction Look-up tables

PROBLEM:
in CTA-MARS the DL@ events are weighted using the 'miss' parameter, which we don't have straight from the DL1 file.

We can calculate it:
_miss__ is just the minimum distance between the true gamma-ray position and the image axis.
we have both the x-axis of the camera and the angle between it and the image axis.

TODO:
* decide if calculating it here or directly from ctapipe/protopipe

#### Produce tables

In [ ]:
lst_optics = OpticsDescription.from_name("LST")
mst_optics = OpticsDescription.from_name("MST")
foc_length_lst = lst_optics.equivalent_focal_length
foc_length_mst = mst_optics.equivalent_focal_length

In [ ]:
# result in meters
disp_lst, miss_lst = camera_to_shower_coordinates(0., 0., data_LST["cog_x"], data_LST["cog_y"], data_LST["psi"])
disp_mst, miss_mst = camera_to_shower_coordinates(0., 0., data_MST["cog_x"], data_MST["cog_y"], data_MST["psi"])
# convert miss parameter from meters to degrees (WARNING: calculation probably approximate or even wrong)
miss_lst_deg = distance_deg(miss_lst, foc_length_lst.value)
miss_mst_deg = distance_deg(miss_mst, foc_length_mst.value)
miss_deg = [miss_lst_deg, miss_mst_deg]

In [ ]:
# build LUTs of Intensity and w/l
H = [None for camera_index in range(len(cameras))]
xedges = [None for camera_index in range(len(cameras))]
yedges = [None for camera_index in range(len(cameras))]
nbins_x = 13
nbins_y = 20
for camera_index in range(len(cameras)):
    H[camera_index], xedges[camera_index], yedges[camera_index] = np.histogram2d(np.log10(DL1[camera_index]["Intensity [#phe]"]),
                     (DL1[camera_index]["Ellipticity"]), bins=[nbins_x,nbins_y], range=[[1.,6.],[0.,1.]])

In [ ]:
# calculate average miss*2
nbins_x = 13
nbins_y = 20
average_miss2 = [np.zeros((13, 20)),np.zeros((13, 20))]
for camera_index in range(len(cameras)):
    for x_bin in range(nbins_x):
        for y_bin in range(nbins_y):
            min_intensity = xedges[camera_index][x_bin]
            max_intensity = xedges[camera_index][x_bin+1]
            min_ellipticity = yedges[camera_index][y_bin]
            max_ellipticity = yedges[camera_index][y_bin+1]
            intensity = DL1[camera_index]["Intensity [#phe]"]
            ellipticity = DL1[camera_index]["Ellipticity"]
            selected_images = (np.log10(intensity) >= min_intensity) & (np.log10(intensity) < max_intensity) & (ellipticity >= min_ellipticity) & (ellipticity < max_ellipticity)
            filtered_miss2 = DL1[camera_index]["miss2 [deg**2]"][selected_images]
            average_miss2[camera_index][x_bin][y_bin] = filtered_miss2.mean()

#### Plot as counts

In [ ]:
nbins_x = 13
nbins_y = 20
cameras = ["LSTCam", "NectarCam"]
LUTs = [None] * len(cameras)

for camera_index in range(len(cameras)):
    
    fig = plt.figure(figsize=(6, 5), tight_layout=False)
    plt.xlabel("log10(Intensity ['phe'])")
    plt.ylabel("Ellipticity [Width [m] / Length [m]]")
    
    LUT= plt.hist2d(np.log10(DL1[camera_index]["Intensity [#phe]"]),
                     DL1[camera_index]["Ellipticity"],
                   bins=[nbins_x, nbins_y],
                   range=[[1.,6.],[0.,1.]],
                   norm=LogNorm(),
                   cmap=plt.cm.rainbow,
                  )
    
    plt.colorbar()
    
    LUTs[camera_index] = LUT
    
    fig.savefig(f"./plots_direction_reconstruction/directionLUTsCounts_{cameras[camera_index]}_protopipe_{config}.png")

#### Plot as weights with miss**2 (deg**2)

In [ ]:
nbins_x = 13
nbins_y = 20
cameras = ["LSTCam", "NectarCam"]
LUTs = [None] * len(cameras)

for camera_index in range(len(cameras)):
    
    fig = plt.figure(figsize=(6, 5), tight_layout=False)
    plt.xlabel("log10(Intensity ['phe'])")
    plt.ylabel("Ellipticity [Width [m] / Length [m]]")
    
    miss2 = np.nan_to_num(average_miss2[camera_index])
    
    LUT = plt.pcolormesh(
                   xedges[camera_index],
                   yedges[camera_index],
                   np.transpose(miss2),
                   cmap=plt.cm.rainbow,
                   norm=LogNorm(),
                  )
    
    plt.colorbar()
    
    LUTs[camera_index] = LUT
    
    fig.savefig(f"./plots_direction_reconstruction/directionLUTsMiss2_{cameras[camera_index]}_protopipe_{config}.png")

In [ ]:
fig = plt.figure(figsize=(6, 5), tight_layout=False)
plt.xlabel("log10(intensity)")
plt.ylabel("miss**2 (deg**2)")

for camera_index in range(len(cameras)):
    
    bincenters = (xedges[camera_index][1:] + xedges[camera_index][:-1])/2
    # bin 9 corresponds to ellipticity between 0.45 and 0.5
    plt.plot(bincenters, np.nan_to_num(average_miss2[camera_index])[:,9], label=f"{cameras[camera_index]}")
    

plt.legend()
    
fig.savefig(f"./plots_direction_reconstruction/directionLUTsMiss2_slice_{cameras[camera_index]}_protopipe_{config}.png")

### Angular resolution (68% containment of the PSF) vs true energy

### Reconstruction efficiency relative to the number of stereoscopic triggers

### Distribution of true core positions for reconstructed events

### Shower core reconstruction

### Shower maximum height reconstruction